In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from collections import defaultdict
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler


from sklearn import model_selection, ensemble
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV, StratifiedKFold
from sklearn.utils.multiclass import unique_labels

# model saving
import pickle 
from joblib import Parallel, delayed
import joblib

# heatmap
import seaborn as sn

from matplotlib import pyplot

In [ ]:
pip install -U --no-cache-dir gdown --pre

In [ ]:
# !gdown 1BQ_5BnEJfpR0a4K1GJB99RVR2Q63eXIF

# data process

In [ ]:
# gene_symbol_mapping= pd.read_csv("./tumor_gene_Symbol_dataset.csv", sep=',')
# gene_symbol_mapping = gene_symbol_mapping.sort_values(by=['Unnamed: 0'])
# gene_symbol_mapping= gene_symbol_mapping.set_index('Unnamed: 0')
# gene_symbol_mapping = gene_symbol_mapping[gene_symbol_mapping['cancer type abbreviation'].notna()]

# gene_symbol_mapping

In [ ]:
# cancer_names= gene_symbol_mapping['cancer type abbreviation'].unique()

In [ ]:
# gene_symbol_mapping['age_at_initial_pathologic_diagnosis']= gene_symbol_mapping['age_at_initial_pathologic_diagnosis'].astype(str)

# d = defaultdict(LabelEncoder)
# columns_to_encode = [
#     "cancer type abbreviation",
#     "age_at_initial_pathologic_diagnosis",
#     "gender",
#     "race"
# ]

# gene_symbol_mapping.loc[:,columns_to_encode] = gene_symbol_mapping.loc[:,columns_to_encode].apply(lambda x: d[x.name].fit_transform(x.fillna('0')))
# gene_symbol_mapping.info()

In [ ]:
# cancer_nums= gene_symbol_mapping['cancer type abbreviation'].unique()
# cancer_name_num_dict = dict(zip(cancer_nums, cancer_names))


In [ ]:
# gene_symbol_mapping_dummies = gene_symbol_mapping.copy()
# gene_symbol_mapping_dummies = pd.get_dummies(gene_symbol_mapping_dummies)

# X = gene_symbol_mapping_dummies.copy()
# y = gene_symbol_mapping_dummies['cancer type abbreviation'].copy()

# load the datasets

In [ ]:
!gdown 15sUdoFB8qTwcsGyPky2CmDwmRi5f5KZP

In [ ]:
# !unzip /content/data.zip
!unzip ./data.zip

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
columns_to_drop = [
    "cancer type abbreviation",
    "age_at_initial_pathologic_diagnosis",
    "gender",
    "race"
]

# 1

In [ ]:
test_split_1= pd.read_csv("./test_split_1.csv", sep=',')
test_split_1

In [ ]:
test_split_1_dummies = test_split_1.copy()
test_split_1_dummies = pd.get_dummies(test_split_1_dummies)

X_test_1 = test_split_1_dummies.drop(columns_to_drop, axis=1).copy()
y_test_1 = test_split_1_dummies['cancer type abbreviation'].copy()

In [ ]:
train_split_1= pd.read_csv("./train_split_1.csv", sep=',')
train_split_1

In [ ]:
train_split_1_dummies = train_split_1.copy()
train_split_1_dummies = pd.get_dummies(train_split_1_dummies)

X_train_1 = train_split_1_dummies.drop(columns_to_drop, axis=1).copy()
y_train_1 = train_split_1_dummies['cancer type abbreviation'].copy()

In [ ]:
X_train1=X_train_1.copy()
y_train1=y_train_1.copy()

In [ ]:
X_train_1,X_valid_1,y_train_1 , y_valid_1  = train_test_split(X_train1,y_train1, test_size=0.2, random_state = 42, stratify=y_train_1)

In [ ]:
sc = MinMaxScaler()
X_train_1 = sc.fit_transform(X_train_1)
X_test_1 = sc.transform(X_test_1)
X_valid_1 = sc.transform(X_valid_1)

In [ ]:
eval_set = [(X_train_1, y_train_1), (X_valid_1, y_valid_1)]

xgb_model_1 = xgb.XGBClassifier(objective="multi:softprob",gpu_id=0,tree_method='gpu_hist', max_depth= 4, random_state=42, 
                                   learning_rate= 0.1, n_estimators=1000,)
# xgb_model= GridSearchCV(estimator=xgb_Classifier, param_grid=grid, cv= 5)
xgb_model_1.fit(X_train_1, y_train_1, verbose=True, early_stopping_rounds=10, eval_metric=["merror", "mlogloss"],eval_set=eval_set)
 

In [ ]:
# xgb_model_1.save_model("xgb_model_1.json")

In [ ]:
# model_xgb_1 = xgb.XGBClassifier()
# model_xgb_1.load_model("/kaggle/input/xgb-model-1/xgb_model_1.json")

In [ ]:
# model_xgb_1.best_ntree_limit


In [ ]:
y_pred_1 = xgb_model_1.predict(X_test_1)

cm1=(confusion_matrix(y_test_1, y_pred_1))


In [ ]:
# def plot_conf_matrix(cm, y_test):
#     labels=unique_labels(y_test)
#     cancer_names=[]
#     for i in labels:
#         if i in cancer_name_num_dict.keys():
#             cancer_names.append(cancer_name_num_dict[i])
    
#     column= [f'Predict_{cancer_name}' for cancer_name in cancer_names]
#     indices= [f'Actual_{cancer_name}' for cancer_name in cancer_names]
    
#     table= pd.DataFrame(cm, columns=column, index=indices)
    
#     fig, ax = pyplot.subplots(figsize=(12,12))         # Sample figsize in inches

#     return sn.heatmap(table, annot=True, fmt="d",  linewidths=.5, ax=ax)

In [ ]:
# plot_conf_matrix(cm1,y_test_1)

In [ ]:
# make predictions for test data
predictions_1 = [round(value) for value in y_pred_1]
# evaluate predictions
accuracy_1 = accuracy_score(y_test_1, predictions_1)
print("Accuracy: %.2f%%" % (accuracy_1 * 100.0))

# precision tp / (tp + fp)
# precision = precision_score(y_test, predictions, average=None)
# print('Precision for every class: %f' % precision.astype(str))
precision_1 = precision_score(y_test_1, predictions_1, average='micro')
print('Precision for all class: %f' % precision_1)

# recall: tp / (tp + fn)
recall_1 = recall_score(y_test_1, predictions_1,average='micro')
print('Recall: %f' % recall_1)
# f1: 2 tp / (2 tp + fp + fn)
f1_1 = f1_score(y_test_1, predictions_1, average='micro')
print('F1 score: %f' % f1_1)

In [ ]:
# !cp -r '/content/tumor_xgb_model_sort_1' /content/gdrive/MyDrive/gene_seq/

# 2

In [ ]:
test_split_2= pd.read_csv("./test_split_2.csv", sep=',')
test_split_2

In [ ]:
test_split_2_dummies = test_split_2.copy()
test_split_2_dummies = pd.get_dummies(test_split_2_dummies)

X_test_2 = test_split_2_dummies.drop(columns_to_drop, axis=1).copy()
y_test_2 = test_split_2_dummies['cancer type abbreviation'].copy()

In [ ]:
train_split_2= pd.read_csv("./train_split_2.csv", sep=',')
# train_split_1

In [ ]:
train_split_2_dummies = train_split_2.copy()
train_split_2_dummies = pd.get_dummies(train_split_2_dummies)

X_train_2 = train_split_2_dummies.drop(columns_to_drop, axis=1).copy()
y_train_2 = train_split_2_dummies['cancer type abbreviation'].copy()

In [ ]:
X_train2=X_train_2.copy()
y_train2=y_train_2.copy()

In [ ]:
X_train_2,X_valid_2,y_train_2 , y_valid_2  = train_test_split(X_train2,y_train2, test_size=0.2, random_state = 42, stratify=y_train_2)

In [ ]:
sc = MinMaxScaler()
X_train_2 = sc.fit_transform(X_train_2)
X_test_2 = sc.transform(X_test_2)
X_valid_2 = sc.transform(X_valid_2)

In [ ]:
eval_set = [(X_train_2, y_train_2), (X_test_2, y_test_2)]

xgb_model_2 = xgb.XGBClassifier(objective="multi:softprob",gpu_id=0,tree_method='gpu_hist', max_depth= 6, random_state=42, 
                                   learning_rate= 0.1, n_estimators=1000)
# xgb_model= GridSearchCV(estimator=xgb_Classifier, param_grid=grid, cv= 5)
xgb_model_2.fit(X_train_2, y_train_2, verbose=True, early_stopping_rounds=10, eval_metric=["merror", "mlogloss"],eval_set=eval_set)
 

In [ ]:
# xgb_model_2.save_model("xgb_model_2.json")
# model_xgb_2 = xgb.XGBClassifier()
# model_xgb_2.load_model("xgb_model_2.json")

In [ ]:
y_pred_2 = xgb_model_2.predict(X_test_2)

cm2=(confusion_matrix(y_test_2, y_pred_2))

In [ ]:
# def plot_conf_matrix():
#     labels=y_test_2.unique()
#     cancer_names=[]
#     for i in labels:
#         if i in cancer_name_num_dict.keys():
#             cancer_names.append(cancer_name_num_dict[i])
    
#     column= [f'Predict_{cancer_name}' for cancer_name in cancer_names]
#     indices= [f'Actual_{cancer_name}' for cancer_name in cancer_names]
    
#     table= pd.DataFrame(cm, columns=column, index=indices)
    
#     fig, ax = pyplot.subplots(figsize=(12,12))         # Sample figsize in inches

#     return sn.heatmap(table, annot=True, fmt="d",  linewidths=.5, ax=ax)

In [ ]:
# plot_conf_matrix(cm2,y_test_2)

In [ ]:
# make predictions for test data
predictions_2 = [round(value) for value in y_pred_2]
# evaluate predictions
accuracy_2 = accuracy_score(y_test_2, predictions_2)
print("Accuracy: %.2f%%" % (accuracy_2 * 100.0))

# precision tp / (tp + fp)
# precision = precision_score(y_test, predictions, average=None)
# print('Precision for every class: %f' % precision.astype(str))
precision_2 = precision_score(y_test_2, predictions_2, average='micro')
print('Precision for all class: %f' % precision_2)

# recall: tp / (tp + fn)
recall_2 = recall_score(y_test_2, predictions_2,average='micro')
print('Recall: %f' % recall_2)
# f1: 2 tp / (2 tp + fp + fn)
f1_2 = f1_score(y_test_2, predictions_2, average='micro')
print('F1 score: %f' % f1_2)

In [ ]:
# !cp -r '/content/tumor_xgb_model_sort_2' /content/gdrive/MyDrive/gene_seq/

# 3

In [ ]:
test_split_3= pd.read_csv("./test_split_3.csv", sep=',')
test_split_3

In [ ]:
test_split_3_dummies = test_split_3.copy()
test_split_3_dummies = pd.get_dummies(test_split_3_dummies)

X_test_3 = test_split_3_dummies.drop(columns_to_drop, axis=1).copy()
y_test_3 = test_split_3_dummies['cancer type abbreviation'].copy()

In [ ]:
train_split_3= pd.read_csv("./train_split_3.csv", sep=',')
# train_split_1

In [ ]:
train_split_3_dummies = train_split_3.copy()
train_split_3_dummies = pd.get_dummies(train_split_3_dummies)

X_train_3 = train_split_3_dummies.drop(columns_to_drop, axis=1).copy()
y_train_3 = train_split_3_dummies['cancer type abbreviation'].copy()

In [ ]:
X_train3=X_train_3.copy()
y_train3=y_train_3.copy()

In [ ]:
X_train_3,X_valid_3,y_train_3 , y_valid_3  = train_test_split(X_train3,y_train3, test_size=0.2, random_state = 42, stratify=y_train_3)

In [ ]:
sc = MinMaxScaler()
X_train_3 = sc.fit_transform(X_train_3)
X_test_3 = sc.transform(X_test_3)
X_valid_3 = sc.transform(X_valid_3)

In [ ]:
eval_set = [(X_train_3, y_train_3), (X_test_3, y_test_3)]

xgb_model_3 = xgb.XGBClassifier(objective="multi:softprob",gpu_id=0,tree_method='gpu_hist', max_depth= 4, random_state=42, 
                                   learning_rate= 0.1, n_estimators=1000,)
# xgb_model= GridSearchCV(estimator=xgb_Classifier, param_grid=grid, cv= 5)
xgb_model_3.fit(X_train_3, y_train_3, verbose=True, early_stopping_rounds=10, eval_metric=["merror", "mlogloss"],eval_set=eval_set)
 

In [ ]:
# xgb_model_3.save_model("xgb_model_3.json")
# model_xgb_3 = xgb.XGBClassifier()
# model_xgb_3.load_model("xgb_model_3.json")

In [ ]:
y_pred_3 = xgb_model_3.predict(X_test_3)

cm3=(confusion_matrix(y_test_3, y_pred_3))

In [ ]:
# def plot_conf_matrix():
#     labels=y_test_3.unique()
#     cancer_names=[]
#     for i in labels:
#         if i in cancer_name_num_dict.keys():
#             cancer_names.append(cancer_name_num_dict[i])
    
#     column= [f'Predict_{cancer_name}' for cancer_name in cancer_names]
#     indices= [f'Actual_{cancer_name}' for cancer_name in cancer_names]
    
#     table= pd.DataFrame(cm, columns=column, index=indices)
    
#     fig, ax = pyplot.subplots(figsize=(12,12))         # Sample figsize in inches

#     return sn.heatmap(table, annot=True, fmt="d",  linewidths=.5, ax=ax)

In [ ]:
# plot_conf_matrix(cm3,y_test_3)

In [ ]:
# make predictions for test data
predictions_3 = [round(value) for value in y_pred_3]
# evaluate predictions
accuracy_3 = accuracy_score(y_test_3, predictions_3)
print("Accuracy: %.2f%%" % (accuracy_3 * 100.0))

# precision tp / (tp + fp)
# precision = precision_score(y_test, predictions, average=None)
# print('Precision for every class: %f' % precision.astype(str))
precision_3 = precision_score(y_test_3, predictions_3, average='micro')
print('Precision for all class: %f' % precision_3)

# recall: tp / (tp + fn)
recall_3 = recall_score(y_test_3, predictions_3,average='micro')
print('Recall: %f' % recall_3)
# f1: 2 tp / (2 tp + fp + fn)
f1_3 = f1_score(y_test_3, predictions_3, average='micro')
print('F1 score: %f' % f1_3)

In [ ]:
# !cp -r '/content/tumor_xgb_model_sort_3' /content/gdrive/MyDrive/gene_seq/

# 4

In [ ]:
test_split_4= pd.read_csv("./test_split_4.csv", sep=',')
test_split_4

In [ ]:
test_split_4_dummies = test_split_4.copy()
test_split_4_dummies = pd.get_dummies(test_split_4_dummies)

X_test_4 = test_split_4_dummies.drop(columns_to_drop, axis=1).copy()
y_test_4 = test_split_4_dummies['cancer type abbreviation'].copy()

In [ ]:
train_split_4= pd.read_csv("./train_split_4.csv", sep=',')
# train_split_1

In [ ]:
train_split_4_dummies = train_split_4.copy()
train_split_4_dummies = pd.get_dummies(train_split_4_dummies)

X_train_4 = train_split_4_dummies.drop(columns_to_drop, axis=1).copy()
y_train_4 = train_split_4_dummies['cancer type abbreviation'].copy()

In [ ]:
X_train4=X_train_4.copy()
y_train4=y_train_4.copy()

In [ ]:
X_train_4,X_valid_4,y_train_4 , y_valid_4  = train_test_split(X_train4,y_train4, test_size=0.2, random_state = 42, stratify=y_train4)

In [ ]:
sc = MinMaxScaler()
X_train_4 = sc.fit_transform(X_train_4)
X_test_4 = sc.transform(X_test_4)
X_valid_4 = sc.transform(X_valid_4)

In [ ]:
eval_set = [(X_train_4, y_train_4), (X_test_4, y_test_4)]

xgb_model_4 = xgb.XGBClassifier(objective="multi:softprob",gpu_id=0,tree_method='gpu_hist', max_depth= 4, random_state=42, 
                                   learning_rate= 0.1, n_estimators=1000)
# xgb_model= GridSearchCV(estimator=xgb_Classifier, param_grid=grid, cv= 5)
xgb_model_4.fit(X_train_4, y_train_4, verbose=False, early_stopping_rounds=10, eval_metric=["merror", "mlogloss"],eval_set=eval_set)
 

In [ ]:
xgb_model_4.save_model("xgb_model_4.json")
model_xgb_4 = xgb.XGBClassifier()
model_xgb_4.load_model("xgb_model_4.json")

In [ ]:
y_pred_4 = xgb_model_4.predict(X_test_4)

cm4=(confusion_matrix(y_test_4, y_pred_4))

In [ ]:
# def plot_conf_matrix():
#     labels=y_test_4.unique()
#     cancer_names=[]
#     for i in labels:
#         if i in cancer_name_num_dict.keys():
#             cancer_names.append(cancer_name_num_dict[i])
    
#     column= [f'Predict_{cancer_name}' for cancer_name in cancer_names]
#     indices= [f'Actual_{cancer_name}' for cancer_name in cancer_names]
    
#     table= pd.DataFrame(cm, columns=column, index=indices)
    
#     fig, ax = pyplot.subplots(figsize=(12,12))         # Sample figsize in inches

#     return sn.heatmap(table, annot=True, fmt="d",  linewidths=.5, ax=ax)

In [ ]:
plot_conf_matrix(cm4,y_test_4)

In [ ]:
# make predictions for test data
predictions_4 = [round(value) for value in y_pred_4]
# evaluate predictions
accuracy_4 = accuracy_score(y_test_4, predictions_4)
print("Accuracy: %.2f%%" % (accuracy_4 * 100.0))

# precision tp / (tp + fp)
# precision = precision_score(y_test, predictions, average=None)
# print('Precision for every class: %f' % precision.astype(str))
precision_4 = precision_score(y_test_4, predictions_4, average='micro')
print('Precision for all class: %f' % precision_4)

# recall: tp / (tp + fn)
recall_4 = recall_score(y_test_4, predictions_4,average='micro')
print('Recall: %f' % recall_4)
# f1: 2 tp / (2 tp + fp + fn)
f1_4 = f1_score(y_test_4, predictions_4, average='micro')
print('F1 score: %f' % f1_4)

In [ ]:
# !cp -r '/content/tumor_xgb_model_sort_4' /content/gdrive/MyDrive/gene_seq/

# 5

In [ ]:
test_split_5= pd.read_csv("./test_split_5.csv", sep=',')
test_split_5

In [ ]:
test_split_5_dummies = test_split_5.copy()
test_split_5_dummies = pd.get_dummies(test_split_5_dummies)

X_test_5 = test_split_5_dummies.drop(columns_to_drop, axis=1).copy()
y_test_5 = test_split_5_dummies['cancer type abbreviation'].copy()

In [ ]:
train_split_5= pd.read_csv("./train_split_5.csv", sep=',')
# train_split_1

In [ ]:
train_split_5_dummies = train_split_5.copy()
train_split_5_dummies = pd.get_dummies(train_split_5_dummies)

X_train_5 = train_split_5_dummies.drop(columns_to_drop, axis=1).copy()
y_train_5 = train_split_5_dummies['cancer type abbreviation'].copy()

In [ ]:
X_train5=X_train_5.copy()
y_train5=y_train_5.copy()

In [ ]:
X_train_5,X_valid_5,y_train_5 , y_valid_5  = train_test_split(X_train5,y_train5, test_size=0.2, random_state = 42, stratify=y_train5)

In [ ]:
sc = MinMaxScaler()
X_train_5 = sc.fit_transform(X_train_5)
X_test_5 = sc.transform(X_test_5)
X_valid_5 = sc.transform(X_valid_5)

In [ ]:
eval_set = [(X_train_5, y_train_5), (X_test_5, y_test_5)]

xgb_model_5 = xgb.XGBClassifier(objective="multi:softprob",gpu_id=0,tree_method='gpu_hist', max_depth= 4, random_state=42, 
                                   learning_rate= 0.1, n_estimators=1000,)
# xgb_model= GridSearchCV(estimator=xgb_Classifier, param_grid=grid, cv= 5)
xgb_model_5.fit(X_train_5, y_train_5, verbose=False, early_stopping_rounds=10, eval_metric=["merror", "mlogloss"],eval_set=eval_set)
 

In [ ]:
# xgb_model_5.save_model("xgb_model_5.json")
# model_xgb_5 = xgb.XGBClassifier()
# model_xgb_5.load_model("xgb_model_5.json")

In [ ]:
y_pred_5 = xgb_model_5.predict(X_test_5)

cm5=(confusion_matrix(y_test_5, y_pred_5))

In [ ]:
# def plot_conf_matrix():
#     labels=y_test_5.unique()
#     cancer_names=[]
#     for i in labels:
#         if i in cancer_name_num_dict.keys():
#             cancer_names.append(cancer_name_num_dict[i])
    
#     column= [f'Predict_{cancer_name}' for cancer_name in cancer_names]
#     indices= [f'Actual_{cancer_name}' for cancer_name in cancer_names]
    
#     table= pd.DataFrame(cm, columns=column, index=indices)
    
#     fig, ax = pyplot.subplots(figsize=(12,12))         # Sample figsize in inches

#     return sn.heatmap(table, annot=True, fmt="d",  linewidths=.5, ax=ax)

In [ ]:
# plot_conf_matrix(cm5, y_test_5)

In [ ]:
# make predictions for test data
predictions_5 = [round(value) for value in y_pred_5]
# evaluate predictions
accuracy_5 = accuracy_score(y_test_5, predictions_5)
print("Accuracy: %.2f%%" % (accuracy_5 * 100.0))

# precision tp / (tp + fp)
# precision = precision_score(y_test, predictions, average=None)
# print('Precision for every class: %f' % precision.astype(str))
precision_5 = precision_score(y_test_5, predictions_5, average='micro')
print('Precision for all class: %f' % precision_5)

# recall: tp / (tp + fn)
recall_5 = recall_score(y_test_5, predictions_5,average='micro')
print('Recall: %f' % recall_5)
# f1: 2 tp / (2 tp + fp + fn)
f1_5 = f1_score(y_test_5, predictions_5, average='micro')
print('F1 score: %f' % f1_5)

In [ ]:
# !cp -r '/content/tumor_xgb_model_sort_5' /content/gdrive/MyDrive/gene_seq/

# avg accuracy

In [ ]:
avg_acc=(accuracy_1+accuracy_2+accuracy_3+accuracy_4+accuracy_5)/5
avg_acc

In [ ]:
cm=cm1+cm2+cm3+cm4+cm5

In [ ]:
def plot_final_conf_matrix():
    labels=y_test_5.unique()
    cancer_names=[]
    for i in labels:
        if i in cancer_name_num_dict.keys():
            cancer_names.append(cancer_name_num_dict[i])
    
    column= [f'Predict_{cancer_name}' for cancer_name in cancer_names]
    indices= [f'Actual_{cancer_name}' for cancer_name in cancer_names]
    
    table= pd.DataFrame(cm, columns=column, index=indices)
    
    fig, ax = pyplot.subplots(figsize=(20,20))         # Sample figsize in inches

    return sn.heatmap(table, annot=True, fmt="d",  linewidths=.5, ax=ax)

In [ ]:
plot_final_conf_matrix()